<h1>XGBoost Cloud Prediction Invocation Template</h1>
<h4>Invoke SageMaker Prediction Service</h4>

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

import boto3
import re
from sagemaker import get_execution_role
import sagemaker

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [ ]:
# SDK 2
# RealTimePredictor renamed to Predictor
# https://sagemaker.readthedocs.io/en/stable/v2.html

# Create a predictor and point to an existing endpoint
endpoint_name = 'xgboost-bikerental-v1'
predictor = sagemaker.predictor.Predictor (endpoint_name=endpoint_name)

In [ ]:
predictor.serializer = CSVSerializer()

In [ ]:
df_all = pd.read_csv('bike_test.csv')

In [ ]:
df_all.head()

In [ ]:
df_all.columns[1:]

In [ ]:
# Need to pass an array to the prediction
# can pass a numpy array or a list of values [[19,1],[20,1]]
arr_test = df_all[df_all.columns[1:]].values

In [ ]:
type(arr_test)

In [ ]:
arr_test.shape

In [ ]:
arr_test[:5]

In [ ]:
result = predictor.predict(arr_test[:2])

In [ ]:
result

In [ ]:
arr_test.shape

### Split the input data into chunks
There are thousands of rows in this data set for which need inference.  
When communicating over internet, it is a good idea to split the data into chunks to prevent payload and timeout error

In [ ]:
# For large number of predictions, we can split the input data and
# Query the prediction service.
# array_split is convenient to specify how many splits are needed
predictions = []
for arr in np.array_split(arr_test,10):
    result = predictor.predict(arr)
    result = result.decode("utf-8")
    result = result.split(',')
    print (arr.shape)
    predictions += [float(r) for r in result]

In [ ]:
len(predictions)

In [ ]:
np.expm1(predictions)

In [ ]:
df_all['count'] = np.expm1(predictions)

In [ ]:
df_all.head()

In [ ]:
df_all[['datetime','count']].to_csv('predicted_count_cloud.csv',index=False)

In [ ]:
# Delete Endpoint to prevent unnecessary charges
predictor.delete_endpoint()